In [2]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

from scipy import stats
import matplotlib.pyplot as plt
from math import *
from datetime import datetime
from datetime import timedelta

%matplotlib inline

In [3]:
def data_process(start,period):
    with open ('sp_500_daily_prices.csv') as data:
        sp500=pd.read_csv(data,usecols=['date','COMNAM','PRC','RET','ASK','BID'],engine='c')
    date=sorted(list(set(sp500['date'])))
    try:
        date.index(start)
    except:
        raise (("Oops! The start date is not contained in our datesets.  Try again..."))
    try:
        train_start=date[-9*period+date.index(start)]
        date[date.index(start)+period]
    except:
        raise ("Oops! The period is too long.  Try again...")
    sp_recent=sp500[sp500['date']>=train_start]
    sp_recent=sp_recent[sp_recent['date']<=date[date.index(start)+period]]
    
    with open ('sp_500_daily_return.csv') as return_data:
        sp_ret=pd.read_csv(return_data,index_col=0,engine='c')
    sp_ret=sp_ret.loc[date[-9*period+date.index(start)]:date[period+date.index(start)]]
    return sp_recent,sp_ret,date

In [4]:
def filter_pair(pvalue_key,num):
    try:
        pvalue_key[num-1]
    except:
        raise("Oops! Too large num")
    selected_pair=[]
    stock_list=[]
    i=0
    while i<num:
        for pair in pvalue_key:
            if pair[1][0] not in stock_list and pair[1][1] not in stock_list:
                selected_pair+=[pair[1]]
                stock_list+=[pair[1][0]]
                stock_list+=[pair[1][1]]
                i+=1
    
    return selected_pair

In [5]:
def find_pair(start,period,num):
    sp_recent,sp_ret,date=data_process(start,period)
    error_return=['E','D','C','B','A','']
    sp_ret=sp_ret.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))
    sp_pca=np.array(sp_ret)
    #print(sp_pca)
    cov_company=np.cov(sp_pca)
    cov_company[np.isnan(cov_company)] = 0.0
    eigen_val,eigen_vector=np.linalg.eig(cov_company)
    eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_ret.columns))
    eigen_vector_max.sort(key=lambda x:x[0],reverse=True)
    eigen_vector_max=list(zip(*eigen_vector_max))
    data=sp_ret[list(eigen_vector_max[1][0:20])]
    pvalue_key=find_cointegrated_pairs(data)
    selected_pair=filter_pair(pvalue_key,num)
    data_inuse=sp_recent[sp_recent['date']>=date[-period+date.index(start)]]
    data_inuse=data_inuse[data_inuse['date']<date[date.index(start)+period]]
    data_inuse.fillna(method='ffill')
    
    companies=set(data_inuse['COMNAM'])
    company_data={}
    for company in companies:
        company_data[company]=data_inuse[data_inuse['COMNAM']==company]
    #print('find_pair_done')
    return company_data,selected_pair

In [6]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    #score_matrix = np.zeros((n, n))
    pvalue_key=[]
    keys = list(data.columns)
    #pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            #score = result[0]
            pvalue = result[1]
            pvalue_key+=[[pvalue,(keys[i],keys[j])]]
            #pairs.append((keys[i], keys[j]))
    pvalue_key=sorted(pvalue_key,key=lambda x:x[0])
    #print('find_cointegrated_pairs_done')
    return pvalue_key

In [7]:
start=20130910
end=20150101
period=60
num_pair=10

In [40]:
data,pairs=find_pair(start,period,num_pair)

/home/xinlu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/xinlu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
signal_exists = ['Y','N']         # exist, not exist
signal_opens  = ['O', 'C']        # open, close
signal_moves  = ['B', 'S']        # buy, sell

In [9]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
#    assert len(Areturn)==len(Breturn),"Wrong data extraction!"
    assert len(priceA)==len(priceB),"Wrong data extraction!"
    Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
    Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]
    
    # regression on return
    beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

    # get the residual epsilon_t
    e_t = np.array(Breturn - beta0 - beta*Areturn)

    Xt=[np.sum(e_t[:i+1]) for i in range(len(Areturn))]

    # regression on X_t
    length = len(Xt)
    Xt_vec = np.array(Xt)
    b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

    # get the residual zeta_t
    z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
    var_z = np.var(z_t)

    # calculate the s-score
    s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

    # trading signal
    if s_score   < -1.25:
        signal_open = signal_opens[0]
        signal_move = signal_moves[0]
    elif s_score > 1.25:
        signal_open = signal_opens[0]
        signal_move = signal_moves[1]
    ### what if s_score=0 so that it satisfy both condition?
    elif s_score < 0.5:
        signal_open = signal_opens[1]
        signal_move = signal_moves[0]
    elif s_score > -0.5:
        signal_open = signal_opens[1]
        signal_move = signal_moves[1]
        ### if a possible error, use try except
    else:
        raise('Warning!')

    return beta,signal_open, signal_move

In [10]:
position_dict = {}  # static variable: 'StockA': position
cash = 50000        # static variable


def build_position(position_dict, cash, pairs, data):
    '''build position for each window
    
    Default parameters
    ------------------
    shares per trade:1000
    transaction cost fee: 0.0005
    '''
    # borrow cost of stock = 0.05 of short position
    borrow_cost = 0.05
    transaction_cost = 0.003
    PNL = []
    
    
    for i in range(60):
        cash_old = cash
        for pair in pairs:
#            Areturn        = data[pair[0]]['RET'].values[i:i+60]
#            Breturn        = data[pair[1]]['RET'].values[i:i+60]
            priceA         = data[pair[0]]['PRC'].values[i:i+60]
            priceB         = data[pair[1]]['PRC'].values[i:i+60]
            tickerA        = pair[0]
            tickerB        = pair[1]
            BidA           = data[pair[0]]['BID'].values
            AskA           = data[pair[0]]['ASK'].values
            BidB           = data[pair[1]]['BID'].values
            AskB           = data[pair[1]]['ASK'].values
            result         = generate_signal(priceA, priceB)
            beta           = result[0]
            open_or_not    = result[1]
            buy_or_sell    = result[2]
            
            # for A
            if (tickerA in position_dict.keys()): 
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash -= (AskA[i+60]+transaction_cost)*abs(position_dict[tickerA])
                        position_dict[tickerA] = 0
                    else:
                        cash += (BidA[i+60]-transaction_cost)*abs(position_dict[tickerA])
                        position_dict[tickerA] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] -= int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += (1-borrow_cost)*(cash/len(pairs))/AskB[i+60]*beta*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] += int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= (cash/len(pairs))/BidB[i+60]*beta*(AskA[i+60]+transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerA] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] = -int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += (1-borrow_cost)*abs(position_dict[tickerA])*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] = int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= abs(position_dict[tickerA])*(AskA[i+60]+transaction_cost)
            
            # for B
            if (tickerB in position_dict.keys()):
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash += (BidB[i+60]-transaction_cost)*abs(position_dict[tickerB])
                        position_dict[tickerB] = 0
                    else:
                        cash -= (AskB[i+60]+transaction_cost)*abs(position_dict[tickerB])
                        position_dict[tickerB] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] += int((cash/len(pairs))/AskB[i+60])
                        cash -= (cash/len(pairs))/AskB[i+60]*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] -= int((cash/len(pairs))/BidB[i+60])
                        cash += (1-borrow_cost)*(cash/len(pairs))/BidB[i+60]*(BidB[i+60]-transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerB] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] = int((cash/len(pairs))/AskB[i+60])
                        cash -= abs(position_dict[tickerB])*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] = int((cash/len(pairs))/BidB[i+60])
                        cash += (1-borrow_cost)*abs(position_dict[tickerB])*(BidB[i+60]-transaction_cost)
        PNL.append(cash - cash_old)    
    return position_dict, cash, PNL

In [41]:
build_position(position_dict, cash, pairs, data)

({'AIRGAS INC': 0,
  'ALTERA CORP': 0,
  'AMGEN INC': 0,
  'AVON PRODUCTS INC': 0,
  'BAKER HUGHES INC': -294,
  'COMERICA INC': 0,
  'DANAHER CORP': 0,
  'GANNETT CO INC': 0,
  'INTERNATIONAL FLAVORS & FRAG INC': 0,
  'INTERNATIONAL PAPER CO': 0,
  'MURPHY OIL CORP': 0,
  'NORTHEAST UTILITIES': 1258,
  'OFFICE DEPOT INC': 0,
  'PROGRESSIVE CORP OH': 0,
  'QUESTAR CORP': 251,
  'RITE AID CORP': 0,
  'TIFFANY & CO NEW': -746,
  'TORCHMARK CORP': 0,
  'WESTERN DIGITAL CORP': 0,
  'WEYERHAEUSER CO': 0},
 169345.36491146404,
 [0,
  -3638.9553704596474,
  -3283.6129903174951,
  -2863.0694830354041,
  -12317.03417942116,
  -1829.0973815613979,
  -4155.1038495099456,
  3463.8272939477356,
  -2528.7941168161597,
  4845.1362696905453,
  -4950.7425615112552,
  512.27350118833783,
  -4063.9799679562238,
  -4930.935380328976,
  -942.5821012786746,
  -2540.7346033555968,
  -5018.5129502558584,
  11448.266511391972,
  -5753.8480996780909,
  5318.2803121924571,
  2167.7375052304524,
  7204.6285615063

In [11]:
def add_date(start_date, add_days):
    year = int(start_date/10000)
    month = int(start_date/100)- year*100
    day = start - year*10000 - month*100
    
    date1 = datetime(year, month, day)
    date2 = date1 + timedelta(days=add_days)
    
    new_date  = date2.year * 10000 + date2.month * 100 + date2.day
    return new_date
    

In [12]:
def backtest(start_date_index, timeline):
    date_list = sorted(list(set(sp500['date'])))
    win_len = 60
    start_date = date_list[start_date_index]
    pair_start_date = date_list[start_date_index -win_len*9]
    
    cash = 50000
    
    #pair_end_date = add_date(start_date, win_len - 1)
    
    
    PNL_list = []
    position_dict = {}
    
    for i in range(timeline):
        print("start: ", pair_start_date)
        data, pairs = find_pair(pair_start_date, win_len)
        position_dict, cash, PNL = build_position(position_dict, cash, pairs, data)
        PNL_list = PNL_list + PNL
        start_date_index += 60
        pair_start_date = date_list[start_date_index - win_len*9]
        print(cash)
    
    #PNL_list = np.array([4200,5400,3500,6300])
    #sp500_index = np.array([100,200,100])
    return_list = np.diff(PNL_list)/PNL_list[:(len(PNL_list)-1)]
    sharp_ratio = (np.mean(return_list) - 0.3) / sqrt(np.var(return_list))
                   
    max_here = pd.expanding_max(return_list)
    dd2here = return_list - max_here
    max_drawdown = dd2here.min()
                   
    beta = np.cov(return_list, sp500_index)[1,0] / np.var(sp500_index)
    
    return PNL_list, sharp_ratio, max_drawdown, beta
      

In [13]:
result = backtest(6000, 5)

NameError: name 'sp500' is not defined

In [ ]:
date_list =sorted(list(set(sp500['date'])))
print(date_list[6000])
print(date_list[6060])